## Question 1. Knowing docker tags

Run the command to get information on Docker 

```docker --help```

Now run the command to get help on the "docker build" command

Which tag has the following text? - *Write the image ID to the file* 

- `--imageid string`
- `--iidfile string`
- `--idimage string`
- `--idfile string`

In [7]:
!docker build --help


Usage:  docker build [OPTIONS] PATH | URL | -

Build an image from a Dockerfile

Options:
      --add-host list           Add a custom host-to-IP mapping (host:ip)
      --build-arg list          Set build-time variables
      --cache-from strings      Images to consider as cache sources
      --disable-content-trust   Skip image verification (default true)
  -f, --file string             Name of the Dockerfile (Default is
                                'PATH/Dockerfile')
      --iidfile string          Write the image ID to the file
      --isolation string        Container isolation technology
      --label list              Set metadata for an image
      --network string          Set the networking mode for the RUN
                                instructions during build (default "default")
      --no-cache                Do not use cache when building the image
  -o, --output stringArray      Output destination (format:
                                type=local,dest=path)
    

#### Answer: `--iidfile string`

## Question 2. Understanding docker first run 

Run docker with the python:3.9 image in an interactive mode and the entrypoint of bash.
Now check the python modules that are installed ( use pip list). 
How many python packages/modules are installed?

- 1
- 6
- 3
- 7

```bash
docker run -it --entrypoint=bash python:3.9
```

Unable to find image 'python:3.9' locally  
3.9: Pulling from library/python  
...  

```bash
pip list
```

Package    Version  
---------- -------  
pip        22.0.4  
setuptools 58.1.0  
wheel      0.38.4  

#### Answer: 3

# Prepare Postgres

Run Postgres and load data as shown in the videos
We'll use the green taxi trips from January 2019:

```wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-01.csv.gz```

You will also need the dataset with zones:

```wget https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv```

Download this data and put it into Postgres (with jupyter notebooks or with a pipeline)


#### Preparing postgres:
```bash
cd ../

#starting pg server and pgadmin via docker-compose
docker-compose up -d

#launching local server for transfering CSV data into docker container (in different terminal)
winpty python -m http.server

#running shell script with dockerized data ingestion
./shell_scripts/ingest_data.sh
```

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from pg_credentials import host, port, db_name, user, password

In [8]:
pg_engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{db_name}')
#conn = engine.connect()

In [6]:
conn.close()

## Question 3. Count records 

How many taxi trips were totally made on January 15?

Tip: started and finished on 2019-01-15. 

Remember that `lpep_pickup_datetime` and `lpep_dropoff_datetime` columns are in the format timestamp (date and hour+min+sec) and not in date.

- 20689
- 20530
- 17630
- 21090

In [7]:
q3_sql = '''
    SELECT
        count(1)
    FROM public.green_taxi_data
    WHERE 
        lpep_pickup_datetime::date = '2019-01-15'
        AND lpep_dropoff_datetime::date = '2019-01-15'
        --AND lpep_pickup_datetime <= lpep_dropoff_datetime
'''

In [15]:
df3 = pd.read_sql(sql=q3_sql, con=pg_engine)

In [16]:
df3.head()

,count
0,20530


#### Answer: 20530

## Question 4. Largest trip for each day

Which was the day with the largest trip distance
Use the pick up time for your calculations.

- 2019-01-18
- 2019-01-28
- 2019-01-15
- 2019-01-10

In [12]:
q4_sql = '''
    SELECT
        lpep_pickup_datetime::date as ride_date,
        max(trip_distance)
    FROM public.green_taxi_data
    WHERE 
        lpep_pickup_datetime::date BETWEEN '2019-01-01' AND '2019-01-31'
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 5
'''

In [17]:
df4 = pd.read_sql(sql=q4_sql, con=pg_engine)

In [18]:
df4.head()

,ride_date,max
0,2019-01-15,117.99
1,2019-01-18,80.96
2,2019-01-28,64.27
3,2019-01-10,64.20
4,2019-01-06,60.91


#### Answer: 2019-01-15

## Question 5. The number of passengers

In 2019-01-01 how many trips had 2 and 3 passengers?
 
- 2: 1282 ; 3: 266
- 2: 1532 ; 3: 126
- 2: 1282 ; 3: 254
- 2: 1282 ; 3: 274

In [19]:
q5_sql = '''
    SELECT
        passenger_count,
        count(1)
    FROM public.green_taxi_data
    WHERE 
        lpep_pickup_datetime::date = '2019-01-01'
        --AND lpep_dropoff_datetime::date = '2019-01-01'
        AND passenger_count IN (2, 3)
    GROUP BY 1
'''

In [20]:
df5 = pd.read_sql(sql=q5_sql, con=pg_engine)

In [22]:
df5.head()

,passenger_count,count
0,2,1282
1,3,254


#### Answer: 2: 1282  3: 254

## Question 6. Largest tip

For the passengers picked up in the Astoria Zone which was the drop off zone that had the largest tip?
We want the name of the zone, not the id.

Note: it's not a typo, it's `tip` , not `trip`

- Central Park
- Jamaica
- South Ozone Park
- Long Island City/Queens Plaza

In [24]:
q6_sql = '''
    SELECT
        doz."Zone",
        max(tip_amount)
    FROM public.green_taxi_data g
    INNER JOIN public.zones puz
        ON g."PULocationID" = puz."LocationID"
        AND puz."Zone" = 'Astoria'
    INNER JOIN public.zones doz
        ON g."DOLocationID" = doz."LocationID"
    GROUP BY 1
    ORDER BY 2 DESC
'''

In [25]:
df6 = pd.read_sql(sql=q6_sql, con=pg_engine)

In [26]:
df6.head()

,Zone,max
0,Long Island City/Queens Plaza,88.00
1,Central Park,30.00
2,Jamaica,25.00
3,None,25.00
4,Astoria,18.16


#### Answer: Long Island City/Queens Plaza